#### 개발 가이드를 참고하고 자신의 api를 발급받아 아래와 같이 사용하면 됨


In [24]:
#카카오 맵 api
# https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord
import requests, json
import numpy as np
from urllib.parse import urlparse
import json
def convertKakaoGeo(addr):
    try:
        REST_API_KEY = '18737d5fc137442681e6094df137a2eb'
        url = "https://dapi.kakao.com/v2/local/search/address.json"
        h = { "Authorization": f"KakaoAK {REST_API_KEY}",'Content-Type': 'application/json' }
        param = {"query":addr} 
        re = requests.get(url,headers = h, params = param).json()
        print(re)
        #위도 경도
        return re['documents'][0]['y'],re['documents'][0]['x'] ,re['documents'][0]['road_address']['building_name']
    except Exception as e:        
        return np.NaN,np.NaN,np.NaN

In [32]:
convertKakaoGeo('서울 강남구 강남대로94길 13 삼경빌딩 1~5층')

{'documents': [{'address': {'address_name': '서울 강남구 역삼동 818-12', 'b_code': '1168010100', 'h_code': '1168064000', 'main_address_no': '818', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_h_name': '역삼1동', 'region_3depth_name': '역삼동', 'sub_address_no': '12', 'x': '127.028419603321', 'y': '37.4995844298918'}, 'address_name': '서울 강남구 강남대로94길 13', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 강남구 강남대로94길 13', 'building_name': '', 'main_building_no': '13', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '역삼동', 'road_name': '강남대로94길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.028419603321', 'y': '37.4995844298918', 'zone_no': '06129'}, 'x': '127.028419603321', 'y': '37.4995844298918'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


('37.4995844298918', '127.028419603321', '')

In [33]:
import folium
lat,lng,name = convertKakaoGeo('서울 강남구 강남대로94길 13 삼경빌딩 1~5층')
m = folium.Map(location=[lat,lng],zoom_start=12)
folium.Marker(
    location=[lat,lng]
    ,tooltip=name
).add_to(m)

{'documents': [{'address': {'address_name': '서울 강남구 역삼동 818-12', 'b_code': '1168010100', 'h_code': '1168064000', 'main_address_no': '818', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_h_name': '역삼1동', 'region_3depth_name': '역삼동', 'sub_address_no': '12', 'x': '127.028419603321', 'y': '37.4995844298918'}, 'address_name': '서울 강남구 강남대로94길 13', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 강남구 강남대로94길 13', 'building_name': '', 'main_building_no': '13', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '역삼동', 'road_name': '강남대로94길', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.028419603321', 'y': '37.4995844298918', 'zone_no': '06129'}, 'x': '127.028419603321', 'y': '37.4995844298918'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [47]:
# 카카오 검색
import pandas as pd
import requests, json
search = '김건희'
REST_API_KEY = '18737d5fc137442681e6094df137a2eb'
url = "https://dapi.kakao.com/v2/search/web"
h = { "Authorization": f"KakaoAK {REST_API_KEY}",'Content-Type': 'application/json' }
result = []
for i in range(1,51):
    param = {"query":search,'page':1,'size':50} 
    re = requests.get(url,headers = h, params = param).json()
    result.append(pd.DataFrame(re['documents']))
df = pd.concat(result).reset_index()

In [92]:
# df.loc[:,'contents']

##### 검색결과 시각화

In [50]:

import json
import re
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
from functools import reduce

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib
import matplotlib.font_manager as fm
font_loc = 'C:/Windows/Fonts/H2PORM.TTF'
font_name = fm.FontProperties(fname=font_loc).get_name()
matplotlib.rc('font',family=font_name)
plt.rcParams['axes.unicode_minus'] = False  # 한글폰트적용시 - 깨짐방지..

In [90]:
def showTextClude(listValue:list, count = 0):
    temp_list = [re.sub(r'[^\w]',' ',w) for w in listValue]
    temp = [i.split() for i in temp_list]
    temp = reduce(lambda x,y : x+y, temp)
    temp = [re.sub(r'[^\w]',' ',w) for w in temp]
    temp =[w for w in temp if len(w) > 1]
    # 불용어 제거
    stopword = ['김건희','윤','석','열','김','건','희','는','을','3','9','34','있다','씨','가','대통령']
    temp = [w for w in temp if not np.any([w[-2:] in stopword, w[-1] in stopword])]
    temp_count = Counter(temp)
    print(temp)
    if count != 0:     
        temp_count = dict(temp_count.most_common(count)) # 상위 count개
    wc = WordCloud(font_loc ,background_color='white',width=800,height=600)
    cloud = wc.generate_from_frequencies(temp_count)
    plt.imshow(cloud)
    plt.axis('off')
    plt.show()

In [89]:
stopworld = ['김건희','윤','석','열','김','건','희','는','을','3','9','34','있다','씨','가','대통령','다']
np.any([w in stopworld for w in '재직했다'])
# '재직했다' in ['김건희','윤','석','열','김','건','희','는','을','3','9','34','있다','씨','가','대통령','다']

True

In [93]:
# df.loc[:,'contents'].values
# showTextClude(df.loc[:,'contents'].values,1000)

In [2]:
# 롯데 시네마 크롤링
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
url = "https://www.lottecinema.co.kr/NLCHS"
driver = webdriver.Chrome('chromedriver')
driver.get(url)
html = driver.page_source  # html 소스 가져오기
soup =  BeautifulSoup(html, 'html.parser')
#nav > ul > li:nth-child(3) > div > ul > li:nth-child(3) > a

[<a href="#" title="경기/인천">경기/인천</a>]

In [3]:
soup.select('#nav > ul > li:nth-child(3) > div > ul > li:nth-child(3) > a')

[<a href="#" title="경기/인천">경기/인천</a>]

In [94]:
# tags = soup.select('#nav > ul > li:nth-child(3) > div > ul > li:nth-child(3) > div > ul > li')
# for i in tags:
#     print(i.text)

In [28]:
# CGV iframe
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
url ='http://www.cgv.co.kr/theaters/'
driver = webdriver.Chrome('chromedriver')
driver.get(url)

# driver.switch_to.frame("#ifrm_movie_time_table")
# html = driver.page_source  # html 소스 가져오기
# soup =  BeautifulSoup(html, 'html.parser')

In [95]:
# iframe일경우 driver를 iframe을 읽을수 있는 형태로 change 한다.
iframe =  driver.find_element(By.CSS_SELECTOR, '.col-detail> iframe')
driver.switch_to.frame(iframe)
html = driver.page_source  # html 소스 가져오기
soup =  BeautifulSoup(html, 'html.parser')

In [35]:
soup.select('.sect-showtimes>ul>li')[0].text

'\n\n\n\n12\n \n                                                정직한 후보2\n\n\n                                            상영중\n\n\n\n\n                                        코미디/ \n                                            107분/ \n                                                2022.09.28\n                                                개봉\n\n\n\n\n\n                                                        2D\n\n                                                        1관 6층\n총\n                                                        158석\n\n\n\n\n16:30잔여좌석148석\n18:50잔여좌석139석\n21:10잔여좌석137석\n23:30잔여좌석151석\n\n\n\n\n\n\n\n                                                        2D\n\n                                                        4관[LCK관] 8층\n총\n                                                        124석\n\n\n\n\n15:10마감\n23:00잔여좌석120석\n\n\n\n\n'

In [36]:
driver.switch_to.default_content()